In [ ]:
# Import necessary libraries
import panel as pn
import pandas as pd
import requests
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import Range1d
from bokeh.models import HoverTool, PanTool, BoxZoomTool, WheelZoomTool, ResetTool

# ----------------------------- Getting data from CoinGecko API -----------------------------

# Function to get CoinGecko coins list
def get_coingecko_coins():
    url = 'https://api.coingecko.com/api/v3/coins/list'
    response = requests.get(url)
    data = response.json()
    coin_dict = {coin['symbol'].upper(): coin['id'] for coin in data}
    return coin_dict

# Fetch the coin list and store it in coin_dict
coin_dict = get_coingecko_coins()

# Define a list of fiat currency codes
currencies = ['USD', 'EUR', 'JPY', 'GBP', 'AUD', 'CAD', 'CHF', 'CNY', 'SEK', 'NZD', 'SGD']

# Function to get historical data
def get_historical_data(coin_id, vs_currency):
    url = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency={vs_currency}&days=max&interval=daily'
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame({'timestamp': [x[0] for x in data['prices']], 'price': [x[1] for x in data['prices']],
                       'market_cap': [x[1] for x in data['market_caps']], 'volume': [x[1] for x in data['total_volumes']]})
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df['daily_change'] = df['price'].pct_change()
    return df

# Function to calculate Simple Moving Average
def calculate_sma(df, window):
    df['sma'] = df['price'].rolling(window=window).mean()
    return df

# Function to calculate Exponential Moving Average
def calculate_ema(df, window):
    df['ema'] = df['price'].ewm(span=window, adjust=False).mean()
    return df

# Function to calculate Bollinger Bands
def calculate_bollinger_bands(df, window):
    df['sma'] = df['price'].rolling(window=window).mean()
    df['std_dev'] = df['price'].rolling(window=window).std()
    df['bollinger_upper'] = df['sma'] + (2 * df['std_dev'])
    df['bollinger_lower'] = df['sma'] - (2 * df['std_dev'])
    return df

# Function to calculate Volume Weighted Average Price (VWAP)
def calculate_vwap(df):
    df['vwap'] = df['volume'].cumsum() / df.index.to_series().diff().dt.total_seconds().cumsum()
    return df

# Function to calculate Relative Strength Index (RSI)
def calculate_rsi(df, window):
    delta = df['price'].diff()
    up = delta.clip(lower=0)
    down = -1*delta.clip(upper=0)
    ma_up = up.rolling(window).mean()
    ma_down = down.rolling(window).mean()
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    df['rsi'] = rsi
    return df

# Function to calculate Moving Average Convergence Divergence (MACD)
def calculate_macd(df, short_window, long_window):
    exp1 = df['price'].ewm(span=short_window, adjust=False).mean()
    exp2 = df['price'].ewm(span=long_window, adjust=False).mean()
    macd = exp1 - exp2
    signal = macd.ewm(span=9, adjust=False).mean()
    df['macdhist'] = macd - signal
    return df

# Define the window size for each indicator
sma_window = 200
ema_window = 200
rsi_window = 14
macd_short_window = 12
macd_long_window = 26

# Define widgets
coin_select = pn.widgets.AutocompleteInput(name='Coin', options=list(coin_dict.keys()) + currencies, value='XRP')
currency_select = pn.widgets.AutocompleteInput(name='Currency', options=list(coin_dict.keys()) + currencies, value='AUD')

# Define the coin_id and vs_currency
coin_id = coin_dict[coin_select.value]
vs_currency = currency_select.value.lower()

# Get historical data
df = get_historical_data(coin_id, vs_currency)

# Calculate SMA
df = calculate_sma(df, sma_window)

# Calculate EMA
df = calculate_ema(df, ema_window)

# Calculate RSI
df = calculate_rsi(df, rsi_window)

# Calculate MACD
df = calculate_macd(df, macd_short_window, macd_long_window)

# Drop NaN values
df = df.dropna()

# ----------------------------- Plotting charts -----------------------------

# Define a HoverTool specific to the 'price' line
hover = HoverTool(
    tooltips=[
        ('date', '@x{%F}'),
        ('price', '@y'),
    ],
    formatters={
        '@x': 'datetime',  # use 'datetime' formatter for '@x' field
    },
    mode='vline'  # display a tooltip whenever the cursor is vertically in line with a glyph
)

# Function to plot closing prices and overlays SMA, EMA and MACD if they are in the DataFrame
def plot_price_and_indicators(df):
    
    # Base plot with closing prices
    p1 = figure(
        x_axis_type="datetime",
        title=f"{coin_id.upper()}",
        plot_height=435,
        plot_width=1445,
        background_fill_color='#D3D3D3',
        tools=[PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool(), hover])
    p1.line(df.index, df['price'], color='green', alpha=0.5, legend_label='Closing Price')

    # Add SMA to plot
    if 'sma' in df.columns:
        p1.line(df.index, df['sma'], color='orange', alpha=0.5, legend_label='SMA')

    # Add EMA to plot
    if 'ema' in df.columns:
        p1.line(df.index, df['ema'], color='red', alpha=0.5, legend_label='EMA')

    # Add MACD to plot
    if 'macdhist' in df.columns:
        p1.line(df.index, df['macdhist'], color='blue', alpha=0.5, legend_label='MACD')

    # Configure plot aesthetics & change other plot attributes to fit with the black background
    p1.title.text_color = 'black'
    p1.xaxis.axis_label_text_color = 'black'
    p1.yaxis.axis_label_text_color = 'black'
    p1.grid.grid_line_color = 'white'
    p1.xaxis.major_label_text_color = 'black'
    p1.yaxis.major_label_text_color = 'black'
    p1.legend.label_text_color = 'black'
    p1.xaxis.axis_label = 'Date'
    p1.yaxis.axis_label = 'Value'
    p1.legend.location = "top_left"

    # Create separate subplot for RSI
    if 'rsi' in df.columns:  # Only add RSI subplot when 'rsi' is in df.columns
        p2 = figure(
            x_axis_type="datetime",
            title="Relative Strength Index",
            plot_height=372,
            plot_width=1445,
            x_range=p1.x_range,
            tools=[PanTool(), BoxZoomTool(), WheelZoomTool(), ResetTool(), hover])
        p2.line(df.index, df['rsi'], color='purple', alpha=0.5)
        p2.grid.grid_line_alpha=0.3
        p2.xaxis.axis_label = 'Date'
        p2.yaxis.axis_label = 'Value'
        p2.y_range = Range1d(0, 100)

        # Combine plots
        p = gridplot([[p1], [p2]])
    else:
        p = p1

    # Return plot
    return pn.pane.Bokeh(p)

# Define widgets
coin_select = pn.widgets.AutocompleteInput(name='Select Crypto:', options=list(coin_dict.keys()) + currencies, value='XRP')
currency_select = pn.widgets.AutocompleteInput(name='Select Currency:', options=list(coin_dict.keys()) + currencies, value='AUD')
sma_select = pn.widgets.Select(name='SMA:', options=[None, 50, 100, 200])
ema_select = pn.widgets.Select(name='EMA:', options=[None, 12, 26, 50, 100])
macd_check = pn.widgets.Checkbox(name='MACD', value=False)
rsi_check = pn.widgets.Checkbox(name='RSI', value=False)
visualise_button = pn.widgets.Button(name='Visualise Plot', button_type='primary')

# Function to display plot
def visualise_plot(event=None):
    global df
    global coin_id
    global vs_currency
    global plot
    
    # Fetch coin id from coin dict
    coin_id = coin_dict[coin_select.value.upper()]
    
    # Ensure that the selected currency is in the correct format, which is uppercase. This vs_currency value will then be used when fetching historical data for the selected cryptocurrency.
    vs_currency = currency_select.value.upper()

    # Get historical data
    df = get_historical_data(coin_id, vs_currency.lower())

    # Calculate indicators
    if sma_select.value:
        df = calculate_sma(df, sma_select.value)
    if ema_select.value:
        df = calculate_ema(df, ema_select.value)
    if rsi_check.value:  # Only calculate RSI when RSI checkbox is checked
        df = calculate_rsi(df, rsi_window)
    if macd_check.value:
        df = calculate_macd(df, macd_short_window, macd_long_window)

    # Drop NaN values
    df = df.dropna()

    # Plot data
    plot = plot_price_and_indicators(df)  # Redraw the plot

    # Update the charts_tab object to include the new plot
    charts_tab.clear()  # Clear the previous contents
    charts_tab.extend([coin_select, currency_select, sma_select, ema_select, rsi_check, macd_check, visualise_button, plot])  # Add the new contents

# Assign the click event handler
visualise_button.on_click(visualise_plot)

# Assign the change event handler for each widget
sma_select.param.watch(visualise_plot, 'value')
ema_select.param.watch(visualise_plot, 'value')
macd_check.param.watch(visualise_plot, 'value')
rsi_check.param.watch(visualise_plot, 'value')

# Combine everything into a column
charts_tab = pn.Column(coin_select, currency_select, sma_select, ema_select, rsi_check, macd_check, visualise_button)

# Display the dashboard in a pop-up window
dashboard = pn.Tabs(('Charts', charts_tab))

# Run the dashboard
dashboard.show()           